> Before Running please adjust to new split strategy to ./workdir/fold_XX

In [1]:
import os
import yaml
from types import SimpleNamespace

yaml_path = "../../config/playground/mpgcn.yaml"

with open(yaml_path, "r") as f:
    cfg = yaml.safe_load(f)

def to_ns(d):
    if isinstance(d, dict):
        return SimpleNamespace(**{k: to_ns(v) for k, v in d.items()})
    elif isinstance(d, list):
        return [to_ns(x) for x in d]
    return d

args = to_ns(cfg)
args.config_path = yaml_path
print("Loaded YAML config.")

Loaded YAML config.


In [2]:
args.work_dir = "../../workdir"
args.dataset_args.root_folder = "../../workdir"
args.dataset_args.object_folder = "../../workdir"

print("Root folder:", args.dataset_args.root_folder)

Root folder: ../../workdir


In [3]:
import torch
import numpy as np
from src.dataset import create as create_dataset
from src.initializer import Initializer
from src.model import create as create_model

In [4]:
import pickle

train_labels_path = os.path.join(args.dataset_args.root_folder, "train_label.pkl")

with open(train_labels_path, "rb") as f:
    train_labels = pickle.load(f)

labels_arr = np.array([l for l, _ in train_labels])
observed_classes = int(labels_arr.max()) + 1

print("Observed label classes:", observed_classes)

Observed label classes: 6


In [5]:
args.dataset_args.debug = True
args.dataset_args.augment = False

# Inject correct class count into model_args
args.model_args.num_class = observed_classes

print("Model will be built with num_class =", args.model_args.num_class)

Model will be built with num_class = 6


In [6]:
feeders, data_shape, num_class, A, parts = create_dataset(
    args.dataset,
    **vars(args.dataset_args)
)

print("train samples:", len(feeders["train"]))
print("eval samples :", len(feeders["eval"]))
print("data shape   :", data_shape)
print("num classes  :", num_class)

train samples: 64
eval samples : 64
data shape   : [4, 6, 48, 96, 1]
num classes  : 6


In [7]:
import torch
from src.model.MPGCN import MPGCN

# Convert adjacency matrix
A_tensor = torch.tensor(A, dtype=torch.float32)
A_tensor = A_tensor.cuda() if torch.cuda.is_available() else A_tensor

# Build the model with ALL required arguments
model = MPGCN(
    data_shape=data_shape,     # shape: [4, 6, 48, 96, 1]
    A=A_tensor,                # adjacency, now torch.Tensor
    parts=parts,               # <-- REQUIRED
    num_class=args.model_args.num_class,
    use_att=args.model_args.use_att,
    kernel_size=args.model_args.kernel_size,
    dilation=args.model_args.dilation,
    reduct_ratio=args.model_args.reduct_ratio
)

model = model.cuda() if torch.cuda.is_available() else model
print("Model successfully built!")

Model successfully built!


In [8]:
sample, label, clip = feeders["train"][0]

x = torch.tensor(sample).float().unsqueeze(0)  # add batch dim
y = torch.tensor([label])

if torch.cuda.is_available():
    x, y = x.cuda(), y.cuda()

print("Input tensor:", x.shape)
print("Label:", y)

Input tensor: torch.Size([1, 4, 6, 48, 96, 1])
Label: tensor([1])


In [9]:
out_tuple = model(x)
logits = out_tuple[0]   # MP-GCN returns (output, feature_maps)

print("Logits shape:", logits.shape)
print("Logits:", logits)

Logits shape: torch.Size([1, 6])
Logits: tensor([[ 0.0183,  0.0206, -0.0080,  0.0081, -0.0122,  0.0369]],
       grad_fn=<AddmmBackward0>)


In [11]:
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.05,
    momentum=0.9,
    nesterov=True,
    weight_decay=0.0002
)

In [12]:
criterion = torch.nn.CrossEntropyLoss()

In [13]:

print("Starting 1-sample overfit test...")

for i in range(200):
    optimizer.zero_grad()

    logits, _ = model(x)
    loss = criterion(logits, y)

    loss.backward()
    optimizer.step()

    pred = logits.argmax(dim=1).item()

    if i % 20 == 0:
        print(f"Iter {i:03d} | Loss = {loss.item():.6f} | Pred = {pred}, True = {y.item()}")

print("DONE.")

Starting 1-sample overfit test...
Iter 000 | Loss = 1.781923 | Pred = 5, True = 1
Iter 020 | Loss = 0.000000 | Pred = 1, True = 1
Iter 040 | Loss = 0.000000 | Pred = 1, True = 1
Iter 060 | Loss = 0.000000 | Pred = 1, True = 1
Iter 080 | Loss = 0.000000 | Pred = 1, True = 1
Iter 100 | Loss = 0.000000 | Pred = 1, True = 1
Iter 120 | Loss = 0.000000 | Pred = 1, True = 1
Iter 140 | Loss = 0.000000 | Pred = 1, True = 1
Iter 160 | Loss = 0.000000 | Pred = 1, True = 1
Iter 180 | Loss = 0.000000 | Pred = 1, True = 1
DONE.
